# **Fine-tuning Mistral 7B Instruct using qLora and Supervise Finetuning on a multi-turn chat dataset**

Welcome to this workshop !

In this workshop, we are going to learn how to fine-tune open an LLM ( Mistral-7B ) using QLoRA on a multi-turn chat dataset.

In our example, we are going to leverage Hugging Face Transformers, TRL, Accelerate, and PEFT.

In Detail you will learn how to:


1.   Setup Development Environment
2.   Load and prepare the dataset
3.   Fine-Tune Mistral 7B with QLoRA


# 1. Setup Development Environment


Before diving into the fine-tuning process, make sure you have the following prerequisites.

GPU: This tutorial can run on a free Google Colab notebook with a GPU. Make sure the GPU is accessible with the following command :

In [1]:
!nvidia-smi

Thu Nov 23 09:42:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   38C    P0              25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

And then install dependencies :

In [2]:
!pip install -q torch #machine learning framework; we just need the datatype
!pip install -q git+https://github.com/huggingface/transformers #huggingface transformers for downloading models weights
!pip install -q datasets #huggingface datasets to download datasets from the hub and manipulate them
!pip install -q peft #Parameter efficient finetuning - for qLora( quantized Low-Rank Adaptation) Finetuning
!pip install -q bitsandbytes #For Model weights quantisation
!pip install -q trl #Transformer Reinforcement Learning - For Finetuning using Supervised Fine-tuning
!pip install -q wandb -U #Used to monitor the model score during training

In [3]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel,prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    DataCollatorForLanguageModeling
)
from trl import SFTTrainer # For supervised finetuning
import bitsandbytes as bnb
##for dataset formating:
from typing import List, Literal, Optional
import random

/home/ubuntu/miniconda/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [19]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

In [20]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True



# 2.   Load and prepare the dataset



For this workshop, we will fine-tune Mistral for a multi-turn chat task.

At Hugging-Face we recently released [Zephyr](https://huggingface.co/collections/HuggingFaceH4/zephyr-7b-6538c6d6d5ddd1cbb1744a66)

We will be using this [dataset](https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k) which was used during the initial training data of zephyr. But since we don't want the training to take forever we'll only sample it to a lower number of examples.

In [21]:
ultrachat_dataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")

N = 2500  # replace with the number of rows you want
ultrachat_dataset = ultrachat_dataset.select(range(N))
ultrachat_dataset

Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 2500
})

In [22]:
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"


def apply_chat_template(
    example, tokenizer, assistant_prefix="<|assistant|>\n"
):

    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )


    return example



In [23]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-v0.1"

# The name of the fine-tuned model. Yes the "I" is on purpose, it's lower grade Zephyr
new_model = "Zhephir-7B-sft"

wandb_project_name = "mistral-7B-MT-sft-workshop"

In [24]:

tokenizer = AutoTokenizer.from_pretrained(
        model_name

    )

if tokenizer.chat_template is None:
    tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
ultrachat_dataset = ultrachat_dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})
ultrachat_dataset


Dataset({
    features: ['prompt', 'prompt_id', 'messages', 'text'],
    num_rows: 2500
})

In [25]:

dataset=ultrachat_dataset.train_test_split(test_size=0.2)

In [26]:
for index in random.sample(range(len(dataset["train"])), 3):
  print(f"#################################  SAMPLE {index} ##################################################")
  print(f"\n\n{dataset['train'][index]['text']} \n\n")

#################################  SAMPLE 1309 ##################################################


<|system|>
</s>
<|user|>
Affiliate Marketing for Events – How Does It Work?
When you’re still new in the business arena, you have to carefully think about what to offer to your customers and promote them in marketing mediums which are convenient for them. You just finished reading Amazon Affiliate Marketing Business Review To Help Your Business Thrive !
Reader Submission affiliate marketing instagram marketingWe've Moved! Update your Reader Now. This feed has moved to: [link] If you haven't already done so, update your reader now with this changed subscription address to get your latest updates from us. Link].
Affiliate marketing is extremely popular these days and can be a great way to get your brand, products, and services in front of wider audiences and drive conversions via leads and sales. The Benefits of Affiliate Marketing [Infographic].
Marketing affiliate marketing event marketi

# 3. Fine-Tune Mistral 7B with QLoRA


In [27]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
      # Auto selects device to put model on.
)

print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [28]:
def find_all_linear_names(model):
    cls = torch.nn.Linear #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    # lm_head is often excluded.
    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


modules = find_all_linear_names(model)
print(modules)


['up_proj', 'down_proj', 'q_proj', 'gate_proj', 'o_proj', 'v_proj', 'k_proj']


In [29]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

#Layers of the NN on which we want to add LoRA Adapters ( it's the ones used in Zephyr)
targets=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
]
#anf these are the all the ones that we can applu LoRA on.
targets=modules

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2.0e-04

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 5

# Log every X updates steps
logging_steps = 1

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map="auto",

In [30]:
# Load the base model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
import os
# You set these environment variables for the HF `Trainer`.
# But you can also define your run beforehand.
os.environ["WANDB_PROJECT"] = wandb_project_name

# Log model when running HF Trainer reporting to wandb.
os.environ["WANDB_LOG_MODEL"] = "true"  # Apparently this is deprecated in version 5 of transformers.

# Use wandb to watch the gradients & model parameters.
os.environ["WANDB_WATCH"] = "all"

In [32]:
run = wandb.init(
    project=wandb_project_name,
    name="train_run_live_workshop",  # Sometimes I use the run name as short descriptor for the run.
    config={
        "split": "train",
        # Optionally, you can add all hyperparameters and configs here for better reproducibility!
    },
    group="train",
    tags=["train", "AdamW"],  # Add tags for what might characterize this run.
    notes="Initial finetuning."
)

In [33]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=targets,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=10, # the number of training steps the model will take
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/home/ubuntu/miniconda/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [34]:
# Train model
trainer.train()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/miniconda/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.022400
2,0.996800
3,1.188600
4,1.027100
5,0.846100
6,1.393100
7,1.166300
8,1.143300
9,1.099500
10,1.543100


/home/ubuntu/miniconda/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=10, training_loss=1.1426365375518799, metrics={'train_runtime': 488.8759, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.02, 'total_flos': 1788747511234560.0, 'train_loss': 1.1426365375518799, 'epoch': 0.02})

In [35]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [36]:
run.finish()

train/epoch,▁▁▅▅▅▅▅████
train/global_step,▁▂▃▃▄▅▆▆▇██
train/learning_rate,▁▁▁▁▁▁▁▁▁▁
train/loss,▃▃▄▃▁▆▄▄▄█
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,0.02
train/global_step,10


In [39]:
inputs = tokenizer(["hello. who are you ?"], return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> hello. who are you ?\n\nI am a 20-year-old student from the Netherlands. I am currently studying at the University of Amsterdam, where I am pursuing a degree in International Business. I am passionate about travel, culture, and exploring new places']

In [38]:
rdm=random.sample(range(len(dataset["test"])), 1)
validation_example=dataset['test'][rdm]['text']
print(validation_example[0])

In [40]:
from transformers import TextStreamer

def stream(user_input):
    prompt=validation_example[0]+"<|user|>"+user_input+"\n</s>\n<|assistant|>"
    runtimeFlag = "cuda:0"
    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=100)

def generate(user_input):
    prompt=validation_example[0]+"<|user|>"+user_input+"\n</s>\n<|assistant|>"
    runtimeFlag = "cuda"
    inputs = tokenizer([prompt], return_tensors="pt")
    generated_ids = model.generate(**inputs,max_new_tokens=100)
    answer=tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return answer

In [41]:
stream("summarize the conversation.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The conversation between the user and the assistant revolved around the HookandBullet website, a platform for fishermen to share their knowledge about fishing. The user asked for instructions on how to access the Hot Spots tab and the Fishing Times charts on the website. The assistant provided step-by-step instructions on how to access these features on the website. The user also asked for clarification 

KeyboardInterrupt: 

In [42]:
# Empty VRAM
import gc
del model
gc.collect()

del trainer
gc.collect()

torch.cuda.empty_cache()
gc.collect()

0